## Adjust to semantic model relationships
This notebook uses Semantic Link Labs to add or alter relationships in a the defined semantic model. 

In [1]:
%pip install semantic-link-labs
import sempy_labs as labs
import sempy.fabric as fabric

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.1/756.1 kB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 183.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 78.7 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-2810f28d-8eaa-475f-babe-1406a56ecb75
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.10.2
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/clust

In [4]:
semanticmodel_name = "AW2020"
workspace_name = "Semantic Showdown"

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 12, Finished, Available, Finished)

## Add relationships to the model
After we added the tables, we need to create relationships to the dimension tables

In [15]:
# setup connection to Tabular Object Model (TOM) and list tables as example
with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=workspace_name) as tom:
    for t in tom.model.Tables:
        print(t.Name)

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 23, Finished, Available, Finished)

FactInternetSales
DimCustomer
DimDate
DimProduct
DimSalesTerritory
DimProductCategory
DimProductSubcategory


In [16]:
# Define relationships as they should be added to the model
# From should always be the many side of the relationship / fact table (current limitation)

relationships = [
    {'FromTable' : 'FactInternetSales', 'FromColumn':'CustomerKey', 'ToTable':'DimCustomer',  'ToColumn':'CustomerKey'},
    {'FromTable' : 'FactInternetSales', 'FromColumn':'DueDateKey', 'ToTable':'DimDate',  'ToColumn':'DateKey'}
]

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 24, Finished, Available, Finished)

In [17]:
# List current existing relationships from the semantic model
currentrelationships = fabric.list_relationships(dataset=semanticmodel_name, workspace=workspace_name)

# Add a new column to store results
currentrelationships["relationship_name"] = ""

# add relationship concatenation in a new column to the dataframe
for idx, row in currentrelationships.iterrows():
        relationship_name = labs.create_relationship_name(
            from_table=row["From Table"],
            from_column=row["From Column"],
            to_table=row["To Table"],
            to_column=row["To Column"]
        )
        currentrelationships.at[idx, "relationship_name"] = relationship_name

# Show the updated DataFrame
display(currentrelationships)

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 702111d5-c78b-4c74-ad47-e0167020cb4a)

In [18]:
# Validating if relationship already exists in the model

for relationship in relationships:
    relationship_name = labs.create_relationship_name(
            from_table=relationship["FromTable"],
            from_column=relationship["FromColumn"],
            to_table=relationship["ToTable"],
            to_column=relationship["ToColumn"]
        )
    # returning results of validation
    does_relationship_exist = relationship_name in currentrelationships['relationship_name'].tolist()
    print(f'Relationship: {relationship_name}; Does it exist? {does_relationship_exist}')
    
    # if relationship does not exist, create relationship
    if not does_relationship_exist:
        with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=workspace_name) as tom:
            tom.add_relationship(
                from_table= relationship["FromTable"], 
                from_column= relationship["FromColumn"], 
                to_table= relationship["ToTable"], 
                to_column= relationship["ToColumn"],
                from_cardinality= 'Many', # ‘Many’, ‘One’, ‘None’
                to_cardinality = 'One', # ‘Many’, ‘One’, ‘None’
                cross_filtering_behavior= 'OneDirection', # ‘OneDirection’, ‘BothDirections’
                is_active= True, 
                security_filtering_behavior= None, 
                rely_on_referential_integrity= False)
            print(f'✅ Relationship {relationship_name} has been created') 
 
 # Rerunning this cell may result in errors, as the cell above (listing the current relationships) forms an input. This input may be cached and therefore return errors. 
 # If the relationship already exists, the error lists something along the lines of "ambiguous path between .... "

StatementMeta(, a315db0d-9430-4fb5-83eb-58861b2f3662, 26, Finished, Available, Finished)

Relationship: 'FactInternetSales'[CustomerKey] -> 'DimCustomer'[CustomerKey]; Does it exist? False
✅ Relationship 'FactInternetSales'[CustomerKey] -> 'DimCustomer'[CustomerKey] has been created
Relationship: 'FactInternetSales'[DueDateKey] -> 'DimDate'[DateKey]; Does it exist? True


TOM Wrapper + Add_Relationship function are documented [here](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.tom.html#sempy_labs.tom.TOMWrapper.add_relationship)